# Finetuning Mô hình trên Dữ liệu DAPT2020

Notebook này thực hiện việc tinh chỉnh (finetuning) một mô hình đã được huấn luyện trước (ví dụ: trên CSE-CIC-IDS2018) bằng cách sử dụng bộ dữ liệu DAPT2020. 

**Lưu ý về Finetuning Random Forest/XGBoost:**
Việc "finetuning" trực tiếp các mô hình này không giống như mạng neural. Các cách tiếp cận có thể bao gồm:
1.  Huấn luyện tiếp mô hình nếu nó hỗ trợ (ví dụ: XGBoost có tham số `xgb_model` để tiếp tục huấn luyện).
2.  Huấn luyện một mô hình mới trên DAPT2020 nhưng sử dụng các siêu tham số tối ưu đã tìm thấy từ mô hình trước đó.
3.  Kết hợp dữ liệu (nếu có thể làm cho đặc trưng và nhãn tương thích) và huấn luyện lại từ đầu.

Trong notebook này, chúng ta sẽ tập trung vào việc tải mô hình XGBoost đã huấn luyện và thử huấn luyện tiếp, và huấn luyện lại Random Forest với cấu hình tương tự trên DAPT2020.

## 1. Thiết lập Môi trường và Import Thư viện

In [1]:
import pandas as pd
import numpy as np
import os
import gc
import time
import joblib # Để tải/lưu model và scaler
import torch # Để tải label_mapping nếu đã lưu bằng torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE # Nếu cần xử lý mất cân bằng trên DAPT2020

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

import matplotlib.pyplot as plt
import seaborn as sns

# Cấu hình đường dẫn (CẦN ĐIỀU CHỈNH CHO PHÙ HỢP)
DAPT2020_DATA_DIR = 'dataset/DAPT-2020/' # Ví dụ: 'dataset/DAPT2020/'
PRETRAINED_MODEL_DIR = 'dataset/working2/' # Thư mục chứa model đã huấn luyện trên CSE-CIC-IDS2018
OUTPUT_DIR = 'dataset/working2/finetuned_DAPT2020/' # Thư mục lưu kết quả finetune

# Tạo thư mục output nếu chưa có
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Hàm trợ giúp (tương tự như trong notebook gốc)
def check_ram():
    import psutil
    process = psutil.Process(os.getpid())
    ram_usage = process.memory_info().rss / 1024 ** 3
    print(f"🔍 Current RAM Usage: {ram_usage:.2f} GB")

def clean_ram():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    print("🧹 RAM and GPU memory cleared (if GPU was used).")
    check_ram()

def evaluate_model_finetuned(model, X_test, y_test, class_names, model_name, results_dir):
    print(f"\n🔍 Bắt đầu đánh giá mô hình đã finetune: {model_name}...")
    start_eval_time = time.time()
    y_pred = model.predict(X_test)
    eval_time = time.time() - start_eval_time
    print(f"   - Thời gian dự đoán: {eval_time:.2f} giây")
    accuracy = accuracy_score(y_test, y_pred) * 100
    report = classification_report(y_test, y_pred, target_names=class_names, zero_division=0)
    print(f"\n🏆 Độ chính xác cuối cùng của {model_name}: {accuracy:.2f}%")
    print(f"\n📊 Báo cáo phân loại (Classification Report) của {model_name}:")
    print(report)
    results_filename = os.path.join(results_dir, f"{model_name.lower().replace(' ', '_').replace('(', '').replace(')', '')}_results.txt")
    with open(results_filename, 'w', encoding='utf-8') as f:
        f.write(f"{model_name} - Final Accuracy: {accuracy:.2f}%\n\n")
        f.write("Classification Report:\n")
        f.write(report)
    print(f"✅ Kết quả đã được lưu vào tệp: '{results_filename}'")
    cm_filename = os.path.join(results_dir, f"{model_name.lower().replace(' ', '_').replace('(', '').replace(')', '')}_confusion_matrix.png")
    plt.figure(figsize=(max(10, len(class_names)), max(8, len(class_names) * 0.8)))
    conf_matrix = confusion_matrix(y_test, y_pred)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 8})
    plt.title(f'Ma trận nhầm lẫn - {model_name}', fontsize=14)
    plt.ylabel('Nhãn Thật', fontsize=12)
    plt.xlabel('Nhãn Dự đoán', fontsize=12)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    plt.yticks(rotation=0, fontsize=10)
    plt.tight_layout()
    plt.savefig(cm_filename, dpi=150)
    print(f"📈 Ma trận nhầm lẫn đã được lưu vào tệp: '{cm_filename}'")
    plt.close()

clean_ram()

🧹 RAM and GPU memory cleared (if GPU was used).
🔍 Current RAM Usage: 0.58 GB


## 2. Tải và Tiền xử lý Cơ bản Dữ liệu DAPT2020

**QUAN TRỌNG:** Bạn cần điều chỉnh đoạn mã dưới đây để đọc đúng các tệp CSV của DAPT2020 và thực hiện các bước làm sạch cơ bản tương tự như đã làm với CSE-CIC-IDS2018.
Các bước bao gồm:
- Đọc dữ liệu (theo chunk nếu lớn).
- Xử lý giá trị thiếu (NaN), vô cực (inf).
- Loại bỏ dòng trùng lặp, header lặp lại.
- Chuyển đổi kiểu dữ liệu cơ bản.

In [2]:
# Import các thư viện cần thiết nếu chúng chưa được import ở đầu notebook
import pandas as pd
import numpy as np
import os
import gc # Để quản lý bộ nhớ
import time # Để đo thời gian thực thi

print("🔄 Bắt đầu tải và tiền xử lý cơ bản DAPT2020 từ file đã gộp...")
start_load_time = time.time()

# --- Đọc dữ liệu DAPT2020 từ file đã gộp ---
# CẬP NHẬT ĐƯỜNG DẪN VÀ TÊN FILE CHO ĐÚNG VỚI FILE GỘP CỦA BẠN
# Biến DAPT2020_DATA_DIR nên được định nghĩa ở cell đầu tiên của notebook
# Ví dụ: DAPT2020_DATA_DIR = 'dataset/DAPT2020/'
# (Giả sử DAPT2020_DATA_DIR đã được định nghĩa ở cell trước đó)
DAPT2020_MERGED_FILE_PATH = os.path.join(DAPT2020_DATA_DIR, 'merged_cleaned.csv')

df_dapt = pd.DataFrame() # Khởi tạo DataFrame rỗng phòng trường hợp lỗi

if not os.path.exists(DAPT2020_MERGED_FILE_PATH):
    print(f"❌ Lỗi: Không tìm thấy file DAPT2020 đã gộp tại: {DAPT2020_MERGED_FILE_PATH}")
    # Cân nhắc việc dừng thực thi hoặc xử lý lỗi phù hợp ở đây nếu file không tồn tại
    # Ví dụ: df_dapt = None hoặc raise FileNotFoundError("File DAPT2020 gộp không tồn tại")
else:
    print(f"   📂 Đang đọc file đã gộp: {DAPT2020_MERGED_FILE_PATH}")
    try:
        # Đọc toàn bộ file CSV đã gộp.
        # Nếu file này vẫn RẤT LỚN (ví dụ, >10-15GB và bạn không đủ RAM),
        # bạn có thể cần đọc theo chunk ngay cả khi nó đã được gộp.
        # df_dapt_chunks = []
        # chunk_iterator = pd.read_csv(DAPT2020_MERGED_FILE_PATH, low_memory=False, chunksize=100000)
        # for chunk in chunk_iterator:
        #     # Thực hiện các bước làm sạch cơ bản cho từng chunk ở đây nếu cần
        #     df_dapt_chunks.append(chunk)
        # df_dapt = pd.concat(df_dapt_chunks, ignore_index=True)
        # del df_dapt_chunks
        # gc.collect()

        # Tạm thời giả định có thể đọc toàn bộ vào bộ nhớ:
        df_dapt = pd.read_csv(DAPT2020_MERGED_FILE_PATH, low_memory=False)
        print(f"      ✅ Đã đọc thành công file đã gộp. Kích thước ban đầu: {df_dapt.shape}")

        # --- Các bước làm sạch cơ bản trên DataFrame đã tải ---
        # 1. Xử lý tên cột (ví dụ: loại bỏ khoảng trắng ở đầu/cuối)
        df_dapt.columns = df_dapt.columns.str.strip()
        print("      - Đã chuẩn hóa tên cột.")

        # 2. Xóa header lặp lại (nếu có trong file gộp - ít khả năng nhưng vẫn kiểm tra)
        # Cần xác định cột đáng tin cậy để kiểm tra header, ví dụ 'Label'
        DAPT_LABEL_COLUMN_NAME = 'Label' # THAY THẾ BẰNG TÊN CỘT NHÃN THỰC TẾ CỦA DAPT2020
        if DAPT_LABEL_COLUMN_NAME in df_dapt.columns:
            initial_rows = len(df_dapt)
            # Đảm bảo so sánh với chuỗi 'Label' nếu header là chuỗi
            df_dapt = df_dapt[df_dapt[DAPT_LABEL_COLUMN_NAME].astype(str) != DAPT_LABEL_COLUMN_NAME]
            if len(df_dapt) < initial_rows:
                print(f"      - Đã loại bỏ {initial_rows - len(df_dapt)} dòng header lặp lại (dựa trên cột '{DAPT_LABEL_COLUMN_NAME}').")
        else:
            print(f"      ⚠️ Cảnh báo: Không tìm thấy cột '{DAPT_LABEL_COLUMN_NAME}' để kiểm tra header lặp lại.")

        # 3. Xử lý giá trị vô cực và NaN
        df_dapt.replace([np.inf, -np.inf], np.nan, inplace=True)
        print("      - Đã xử lý giá trị vô cực.")

        # 4. Chuyển các cột số học tiềm năng sang kiểu số, lỗi ép thành NaN
        #    Sau đó fill NaN bằng 0 cho các cột số này.
        #    Cần xác định các cột không phải số của DAPT2020
        #    (ví dụ: 'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp', và cột nhãn)
        #    LƯU Ý: Cột 'Protocol' và 'Timestamp' sẽ được xử lý/loại bỏ ở các bước sau nếu cần.
        potential_non_numeric_dapt = [
            'Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port',
            'Protocol', 'Timestamp', DAPT_LABEL_COLUMN_NAME # Sử dụng biến đã định nghĩa ở trên
            # Thêm bất kỳ cột nào khác bạn biết chắc chắn không phải là số ở đây
        ]
        # Loại bỏ các cột không tồn tại khỏi danh sách này để tránh lỗi
        actual_potential_non_numeric = [col for col in potential_non_numeric_dapt if col in df_dapt.columns]

        numeric_cols_to_convert = df_dapt.columns.difference(actual_potential_non_numeric, sort=False)

        print(f"      - Đang chuyển đổi {len(numeric_cols_to_convert)} cột sang kiểu số và fill NaN bằng 0...")
        for col in numeric_cols_to_convert:
            if col in df_dapt.columns: # Đảm bảo cột tồn tại
                df_dapt[col] = pd.to_numeric(df_dapt[col], errors='coerce')
                # Chỉ fill NaN nếu cột đó thực sự có giá trị NaN sau khi coerce
                if df_dapt[col].isnull().any():
                    df_dapt[col].fillna(0, inplace=True)
        print("      - Chuyển đổi kiểu số và fill NaN hoàn tất.")

        # 5. Xóa các dòng hoàn toàn rỗng (nếu còn sau các bước trên)
        initial_rows = len(df_dapt)
        df_dapt.dropna(how='all', inplace=True)
        if len(df_dapt) < initial_rows:
            print(f"      - Đã loại bỏ {initial_rows - len(df_dapt)} dòng hoàn toàn rỗng.")

        # 6. Xóa các dòng trùng lặp tổng thể
        initial_rows = len(df_dapt)
        df_dapt.drop_duplicates(inplace=True)
        if len(df_dapt) < initial_rows:
            print(f"      - Đã loại bỏ {initial_rows - len(df_dapt)} dòng trùng lặp.")

        print(f"✅ DAPT2020 sau khi làm sạch cơ bản. Kích thước: {df_dapt.shape}")

    except FileNotFoundError:
        print(f"      ❌ Lỗi: Không tìm thấy tệp - {DAPT2020_MERGED_FILE_PATH}")
        df_dapt = pd.DataFrame() # Đảm bảo df_dapt là DataFrame rỗng nếu lỗi
    except pd.errors.EmptyDataError:
        print(f"      ⚠️ Cảnh báo: Tệp rỗng - {DAPT2020_MERGED_FILE_PATH}")
        df_dapt = pd.DataFrame() # Đảm bảo df_dapt là DataFrame rỗng nếu lỗi
    except Exception as e:
        print(f"      ❌ Lỗi khi xử lý {DAPT2020_MERGED_FILE_PATH}: {e}")
        df_dapt = pd.DataFrame() # Đảm bảo df_dapt là DataFrame rỗng nếu lỗi

if df_dapt.empty:
   print("❌ Không có dữ liệu DAPT2020 nào được tải thành công. Kiểm tra lại đường dẫn và nội dung file.")
   # Cân nhắc việc dừng notebook ở đây nếu không có dữ liệu
   # import sys
   # sys.exit("Dừng thực thi do không tải được dữ liệu DAPT2020.")

load_time = time.time() - start_load_time
print(f"✅ Tải và tiền xử lý cơ bản DAPT2020 hoàn thành trong {load_time:.2f} giây.")

# Gọi hàm check_ram() đã được định nghĩa ở cell đầu tiên của notebook
# (Giả sử hàm check_ram() được định nghĩa ở cell "1. Thiết lập Môi trường và Import Thư viện")
if 'check_ram' in locals() and callable(check_ram):
    check_ram()
else:
    print("Hàm check_ram() chưa được định nghĩa.")

🔄 Bắt đầu tải và tiền xử lý cơ bản DAPT2020 từ file đã gộp...
   📂 Đang đọc file đã gộp: dataset/DAPT-2020/merged_cleaned.csv
      ✅ Đã đọc thành công file đã gộp. Kích thước ban đầu: (165154, 85)
      - Đã chuẩn hóa tên cột.
      ⚠️ Cảnh báo: Không tìm thấy cột 'Label' để kiểm tra header lặp lại.
      - Đã xử lý giá trị vô cực.
      - Đang chuyển đổi 78 cột sang kiểu số và fill NaN bằng 0...


C:\Users\linhv\AppData\Local\Temp\ipykernel_19084\1002639517.py:85: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_dapt[col].fillna(0, inplace=True)


      - Chuyển đổi kiểu số và fill NaN hoàn tất.
      - Đã loại bỏ 82577 dòng trùng lặp.
✅ DAPT2020 sau khi làm sạch cơ bản. Kích thước: (82577, 85)
✅ Tải và tiền xử lý cơ bản DAPT2020 hoàn thành trong 2.89 giây.
🔍 Current RAM Usage: 0.64 GB


## 3. Ánh xạ Đặc trưng và Nhãn, Chuẩn hóa Dữ liệu DAPT2020

Đây là bước **CỰC KỲ QUAN TRỌNG**.
1.  **Tải danh sách đặc trưng và scaler** đã được sử dụng/tạo ra từ mô hình huấn luyện trên CSE-CIC-IDS2018.
2.  **Đảm bảo DAPT2020 có các đặc trưng tương ứng.** Nếu thiếu, bạn cần xem xét việc tạo ra chúng hoặc loại bỏ khỏi cả hai bộ dữ liệu (nếu không thể tạo).
3.  **Ánh xạ nhãn tấn công của DAPT2020** sang một hệ thống nhãn nhất quán. Nếu bạn muốn mô hình cũ dự đoán các lớp cũ, bạn cần ánh xạ các tấn công của DAPT2020 vào các lớp đó, hoặc tạo lớp "unknown/other attack". Nếu bạn muốn mô hình học các tấn công mới từ DAPT2020, bạn cần định nghĩa lại `label_mapping` và có thể cấu trúc lại lớp output của mô hình (khó hơn với RF/XGBoost đã lưu).
4.  **Chuẩn hóa dữ liệu DAPT2020** sử dụng scaler đã được `fit` trên dữ liệu huấn luyện CSE-CIC-IDS2018.

In [3]:
print("🔄 Bắt đầu ánh xạ đặc trưng, nhãn và chuẩn hóa DAPT2020...")

# --- 3.1 Tải Scaler và Danh sách Đặc trưng từ mô hình CSE-CIC-IDS2018 --- 
try:
    PRETRAINED_SCALER_PATH = os.path.join(PRETRAINED_MODEL_DIR, 'scaler.pkl')
    scaler_cse = joblib.load(PRETRAINED_SCALER_PATH)
    print(f"✅ Scaler từ CSE-CIC-IDS2018 đã được tải từ: {PRETRAINED_SCALER_PATH}")
except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file scaler tại '{PRETRAINED_SCALER_PATH}'. Hãy đảm bảo nó đã được lưu từ notebook huấn luyện gốc.")
    # exit()
except Exception as e:
    print(f"❌ Lỗi khi tải scaler: {e}")
    # exit()

try:
    # Giả sử scaler.pkl lưu trữ thông tin về các feature_names mà nó đã được fit
    # Nếu scaler của bạn là MinMaxScaler từ sklearn, nó có thuộc tính 'feature_names_in_'
    if hasattr(scaler_cse, 'feature_names_in_'):
        expected_features_cse = scaler_cse.feature_names_in_.tolist()
        print(f"✅ Danh sách {len(expected_features_cse)} đặc trưng kỳ vọng từ scaler CSE-CIC-IDS2018 đã được tải.")
        # print(expected_features_cse) # Bỏ comment để xem
    else:
        # Nếu scaler không có feature_names_in_, bạn cần tải danh sách đặc trưng từ file riêng
        FEATURE_LIST_PATH = os.path.join(PRETRAINED_MODEL_DIR, 'feature_list.txt') # Giả sử bạn đã lưu nó
        with open(FEATURE_LIST_PATH, 'r') as f:
            expected_features_cse = [line.strip() for line in f]
        print(f"✅ Danh sách {len(expected_features_cse)} đặc trưng kỳ vọng từ file '{FEATURE_LIST_PATH}' đã được tải.")
except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file danh sách đặc trưng 'feature_list.txt' tại '{PRETRAINED_MODEL_DIR}'.")
    print("      Vui lòng tạo file này từ notebook huấn luyện gốc hoặc đảm bảo scaler lưu trữ tên đặc trưng.")
    expected_features_cse = [] # Khởi tạo rỗng để tránh lỗi sau này, nhưng đây là vấn đề cần sửa
    # exit()
except Exception as e:
    print(f"❌ Lỗi khi tải danh sách đặc trưng: {e}")
    expected_features_cse = []
    # exit()

# --- 3.2 Xử lý Đặc trưng trong DAPT2020 --- 
if 'df_dapt' not in locals() or df_dapt.empty:
    print("❌ DataFrame DAPT2020 (df_dapt) rỗng hoặc chưa được tạo. Không thể tiếp tục.")
    # exit()
else:
    print("   Đang chuẩn bị đặc trưng cho DAPT2020...")
    df_dapt_processed = df_dapt.copy() # Làm việc trên bản sao

    # Loại bỏ các cột không có trong expected_features_cse (sau khi xử lý Protocol, Label)
    # và thêm các cột bị thiếu từ expected_features_cse (điền giá trị 0 hoặc NaN rồi fill)

    # --- 3.2.1 Xử lý cột Protocol (One-Hot Encoding) --- 
    # Giả sử DAPT2020 cũng có cột 'Protocol'. Cần đảm bảo nó được mã hóa tương tự CSE-CIC-IDS2018.
    # Các cột Protocol one-hot kỳ vọng từ CSE-CIC-IDS2018 (ví dụ: 'Protocol_0', 'Protocol_6', 'Protocol_17')
    expected_protocol_cols = [col for col in expected_features_cse if col.startswith('Protocol_')]

    if 'Protocol' in df_dapt_processed.columns:
        print("      Mã hóa One-Hot cho cột 'Protocol' của DAPT2020...")
        # Chuyển sang category để get_dummies hoạt động đúng với các giá trị số của protocol
        df_dapt_processed['Protocol'] = df_dapt_processed['Protocol'].astype('category')
        df_dapt_processed = pd.get_dummies(df_dapt_processed, columns=['Protocol'], prefix='Protocol', dtype=np.int8)
        print(f"         Các cột Protocol mới: {[col for col in df_dapt_processed.columns if col.startswith('Protocol_')]}")
    else:
        print("      ⚠️ Không tìm thấy cột 'Protocol' trong DAPT2020. Sẽ thêm các cột Protocol_x kỳ vọng với giá trị 0.")
    
    # Đảm bảo tất cả các cột Protocol_x từ CSE-CIC-IDS2018 đều có trong DAPT2020 (nếu thiếu thì thêm và fill 0)
    for p_col in expected_protocol_cols:
        if p_col not in df_dapt_processed.columns:
            print(f"         Thêm cột Protocol bị thiếu: {p_col} với giá trị 0.")
            df_dapt_processed[p_col] = np.int8(0)

    # --- 3.2.2 Ánh xạ Nhãn DAPT2020 --- 
    # Đây là phần CỰC KỲ QUAN TRỌNG và phụ thuộc vào mục tiêu của bạn.
    # GIẢ ĐỊNH: Bạn muốn ánh xạ các nhãn của DAPT2020 vào hệ thống nhãn của CSE-CIC-IDS2018.
    # Nếu một loại tấn công trong DAPT2020 không có trong CSE, bạn có thể map nó vào 'Benign' hoặc một lớp 'OtherAttack'.
    print("      Đang ánh xạ nhãn cho DAPT2020...")
    try:
        LABEL_MAPPING_CSE_PATH = os.path.join(PRETRAINED_MODEL_DIR, 'label_mapping.pth')
        label_mapping_cse = torch.load(LABEL_MAPPING_CSE_PATH)
        # Đảo ngược mapping để lấy tên lớp từ số
        # class_names_cse = [k for k, v in sorted(label_mapping_cse.items(), key=lambda item: item[1])]
        print(f"         Label mapping của CSE-CIC-IDS2018: {label_mapping_cse}")
    except FileNotFoundError:
        print(f"❌ Lỗi: Không tìm thấy tệp label_mapping.pth của CSE-CIC-IDS2018 tại {LABEL_MAPPING_CSE_PATH}")
        print("      Không thể ánh xạ nhãn DAPT2020 một cách đáng tin cậy.")
        label_mapping_cse = {"Benign": 0} # Fallback rất cơ bản
        # exit()
    except Exception as e:
        print(f"❌ Lỗi khi tải label_mapping.pth của CSE: {e}")
        label_mapping_cse = {"Benign": 0}
        # exit()

    # **** BẠN CẦN ĐỊNH NGHĨA ÁNH XẠ NÀY CHO DAPT2020 ****
    # Ví dụ: Giả sử DAPT2020 có các nhãn 'Normal', 'PortScan', 'WebAppAttack'
    # và bạn muốn map 'PortScan' vào một lớp tương tự trong CSE (nếu có) hoặc một lớp chung.
    # Đây chỉ là VÍ DỤ, bạn cần thay thế bằng các nhãn thực tế của DAPT2020.
    dapt_to_cse_label_map = {
        'Normal': label_mapping_cse.get('Benign', 0), # Ánh xạ Normal của DAPT2020 vào Benign của CSE
        'Benign': label_mapping_cse.get('Benign', 0),
        # Ví dụ nếu DAPT2020 có 'PortScan' và CSE có một lớp cho DoS/Scan
        # 'PortScan_DAPT': label_mapping_cse.get('DoS attacks-Hulk', 0), # VÍ DỤ THÔI!
        # Nếu không có lớp tương ứng, có thể tạo một lớp mới (nếu mô hình hỗ trợ) hoặc map vào Benign/Other.
        # Trong trường hợp finetuning mô hình đã có, việc thêm lớp mới không đơn giản với RF/XGB.
    }
    # Lấy tên cột nhãn của DAPT2020 (ví dụ: 'Label' hoặc 'Attack_Category')
    DAPT_LABEL_COLUMN = 'Label' # THAY THẾ NẾU CẦN
    if DAPT_LABEL_COLUMN not in df_dapt_processed.columns:
        print(f"❌ Lỗi: Không tìm thấy cột nhãn '{DAPT_LABEL_COLUMN}' trong DAPT2020.")
        # exit()
    else:
        # Loại bỏ khoảng trắng ở cột nhãn DAPT2020
        df_dapt_processed[DAPT_LABEL_COLUMN] = df_dapt_processed[DAPT_LABEL_COLUMN].astype(str).str.strip()
        print(f"         Các nhãn gốc trong DAPT2020: {df_dapt_processed[DAPT_LABEL_COLUMN].unique()}")
        
        # Ánh xạ nhãn, những nhãn không có trong map sẽ thành NaN, sau đó fill bằng giá trị default (ví dụ: Benign)
        default_label_value = label_mapping_cse.get('Benign', 0)
        df_dapt_processed['Label_mapped'] = df_dapt_processed[DAPT_LABEL_COLUMN].map(dapt_to_cse_label_map)
        unmapped_count = df_dapt_processed['Label_mapped'].isnull().sum()
        if unmapped_count > 0:
            print(f"         ⚠️ {unmapped_count} nhãn trong DAPT2020 không được ánh xạ, sẽ được gán là Benign (hoặc giá trị default). Hãy kiểm tra dapt_to_cse_label_map!")
            df_dapt_processed['Label_mapped'].fillna(default_label_value, inplace=True)
        
        df_dapt_processed['Label'] = df_dapt_processed['Label_mapped'].astype(int) # Sử dụng cột Label chuẩn
        df_dapt_processed.drop(columns=['Label_mapped', DAPT_LABEL_COLUMN], inplace=True, errors='ignore')
        print(f"         Các nhãn sau khi ánh xạ và fill: {df_dapt_processed['Label'].unique()}")
    
    # --- 3.2.3 Đồng bộ hóa các cột đặc trưng --- 
    # Giữ lại các cột 'Label' và các cột có trong expected_features_cse
    # expected_features_cse là danh sách các tên cột mà scaler và mô hình gốc đã được huấn luyện trên đó,
    # không bao gồm cột Protocol gốc, nhưng bao gồm các cột Protocol_x đã one-hot.
    current_dapt_features = df_dapt_processed.columns.difference(['Label'], sort=False).tolist()

    final_feature_set_for_dapt = []
    for ef_col in expected_features_cse: # Duyệt theo thứ tự của scaler gốc
        if ef_col in current_dapt_features:
            final_feature_set_for_dapt.append(ef_col)
        else:
            print(f"      ⚠️ Đặc trưng kỳ vọng '{ef_col}' không có trong DAPT2020 đã xử lý. Sẽ thêm cột này với giá trị 0.")
            df_dapt_processed[ef_col] = 0 # Thêm cột thiếu và fill 0
            final_feature_set_for_dapt.append(ef_col)

    # Loại bỏ các cột trong DAPT2020 không có trong expected_features_cse
    cols_to_drop_from_dapt = [col for col in current_dapt_features if col not in expected_features_cse]
    if cols_to_drop_from_dapt:
        print(f"      Loại bỏ {len(cols_to_drop_from_dapt)} cột từ DAPT2020 không có trong danh sách đặc trưng kỳ vọng: {cols_to_drop_from_dapt[:5]}...")
        df_dapt_processed.drop(columns=cols_to_drop_from_dapt, inplace=True, errors='ignore')
    
    # Sắp xếp lại các cột của DAPT2020 theo đúng thứ tự của expected_features_cse
    # và thêm cột Label vào cuối
    label_column_dapt = df_dapt_processed.pop('Stage')
    df_dapt_processed = df_dapt_processed[expected_features_cse] # Đảm bảo đúng thứ tự và chỉ các cột cần thiết
    df_dapt_processed['Stage'] = label_column_dapt

    print(f"      ✅ DAPT2020 đã được điều chỉnh để có {len(expected_features_cse)} đặc trưng, shape: {df_dapt_processed.shape}")

    # --- 3.3 Chuẩn hóa Dữ liệu DAPT2020 bằng Scaler của CSE-CIC-IDS2018 --- 
    if 'scaler_cse' in locals() and expected_features_cse:
        print("      Chuẩn hóa dữ liệu DAPT2020 sử dụng scaler đã fit trên CSE-CIC-IDS2018...")
        X_dapt = df_dapt_processed.drop(columns=['Label'])
        y_dapt = df_dapt_processed['Label']
        
        # Đảm bảo X_dapt chỉ chứa các cột mà scaler đã được fit
        X_dapt_scaled_np = scaler_cse.transform(X_dapt[expected_features_cse]) # Sử dụng đúng expected_features_cse
        X_dapt_scaled = pd.DataFrame(X_dapt_scaled_np, columns=expected_features_cse, index=X_dapt.index)
        
        df_dapt_final = X_dapt_scaled.copy()
        df_dapt_final['Label'] = y_dapt.values
        
        print(f"         Phạm vi đặc trưng DAPT2020 sau khi chuẩn hóa: {X_dapt_scaled_np.min():.4f} đến {X_dapt_scaled_np.max():.4f}")
        print(f"      ✅ Chuẩn hóa DAPT2020 hoàn tất. Shape cuối cùng: {df_dapt_final.shape}")
    else:
        print("      ❌ Không thể chuẩn hóa DAPT2020 do thiếu scaler hoặc danh sách đặc trưng kỳ vọng.")
        df_dapt_final = df_dapt_processed.copy() # Sử dụng dữ liệu chưa chuẩn hóa nếu lỗi

del df_dapt, df_dapt_processed, X_dapt, X_dapt_scaled # Giải phóng bộ nhớ
gc.collect()
check_ram()

🔄 Bắt đầu ánh xạ đặc trưng, nhãn và chuẩn hóa DAPT2020...
✅ Scaler từ CSE-CIC-IDS2018 đã được tải từ: dataset/working2/scaler.pkl
✅ Danh sách 72 đặc trưng kỳ vọng từ scaler CSE-CIC-IDS2018 đã được tải.
   Đang chuẩn bị đặc trưng cho DAPT2020...
      Mã hóa One-Hot cho cột 'Protocol' của DAPT2020...
         Các cột Protocol mới: ['Protocol_0', 'Protocol_6', 'Protocol_17']
      Đang ánh xạ nhãn cho DAPT2020...
         Label mapping của CSE-CIC-IDS2018: {'Benign': 0, 'Bot': 1, 'DDOS attack-HOIC': 2, 'DDOS attack-LOIC-UDP': 3, 'DoS attacks-GoldenEye': 4, 'DoS attacks-Hulk': 5, 'DoS attacks-SlowHTTPTest': 6, 'DoS attacks-Slowloris': 7, 'FTP-BruteForce': 8, 'Infilteration': 9, 'SSH-Bruteforce': 10, 'Brute Force -Web': 11, 'Brute Force -XSS': 12, 'SQL Injection': 13}
❌ Lỗi: Không tìm thấy cột nhãn 'Label' trong DAPT2020.
      ⚠️ Đặc trưng kỳ vọng 'CWE Flag Count' không có trong DAPT2020 đã xử lý. Sẽ thêm cột này với giá trị 0.
      Loại bỏ 16 cột từ DAPT2020 không có trong danh sách đặc

KeyError: 'Stage'

## 4. Phân chia Dữ liệu DAPT2020 và Xử lý Mất cân bằng (Nếu cần)

In [ ]:
if 'df_dapt_final' not in locals() or df_dapt_final.empty:
    print("❌ DataFrame DAPT2020 cuối cùng (df_dapt_final) rỗng hoặc chưa được tạo. Không thể tiếp tục.")
    # exit()
else:
    print(f"🔄 Phân chia dữ liệu DAPT2020 (shape: {df_dapt_final.shape})...")
    # Kiểm tra xem cột 'Label' có tồn tại không
    if 'Label' not in df_dapt_final.columns:
        print("❌ Lỗi: Cột 'Label' không tồn tại trong df_dapt_final.")
        # exit()
    else:
        # Kiểm tra số lượng mẫu của mỗi lớp trước khi stratify
        label_counts_dapt = df_dapt_final['Label'].value_counts()
        print(f"   Phân bổ nhãn trong DAPT2020 trước khi chia: \n{label_counts_dapt}")
        
        # Loại bỏ các lớp có ít hơn 2 mẫu (điều kiện cho stratify)
        valid_labels_for_stratify = label_counts_dapt[label_counts_dapt >= 2].index
        df_dapt_stratify_ready = df_dapt_final[df_dapt_final['Label'].isin(valid_labels_for_stratify)]
        
        if len(df_dapt_stratify_ready) < len(df_dapt_final):
            print(f"   ⚠️ Đã loại bỏ {len(df_dapt_final) - len(df_dapt_stratify_ready)} mẫu thuộc các lớp có ít hơn 2 mẫu để thực hiện stratify.")

        if not df_dapt_stratify_ready.empty and len(df_dapt_stratify_ready['Label'].unique()) > 1:
            df_train_dapt, df_test_dapt = train_test_split(
                df_dapt_stratify_ready, 
                test_size=0.2, # Tỷ lệ tập kiểm thử
                random_state=42, 
                stratify=df_dapt_stratify_ready['Label']
            )
            print(f"   ✅ DAPT2020 đã chia: Train shape: {df_train_dapt.shape}, Test shape: {df_test_dapt.shape}")
        elif not df_dapt_stratify_ready.empty:
             print("   ⚠️ Dữ liệu DAPT2020 chỉ còn một lớp duy nhất sau khi lọc, không thể stratify. Sử dụng train_test_split thông thường.")
             df_train_dapt, df_test_dapt = train_test_split(df_dapt_stratify_ready, test_size=0.2, random_state=42)
             print(f"   ✅ DAPT2020 đã chia: Train shape: {df_train_dapt.shape}, Test shape: {df_test_dapt.shape}")
        else:
            print("   ❌ Không đủ dữ liệu DAPT2020 để chia sau khi lọc các lớp ít mẫu.")
            df_train_dapt, df_test_dapt = pd.DataFrame(), pd.DataFrame()
            # exit()

        # (Tùy chọn) Xử lý mất cân bằng trên df_train_dapt nếu cần
        print("\n   Kiểm tra và xử lý mất cân bằng cho tập huấn luyện DAPT2020 (nếu cần)...")
        label_counts_train_dapt = df_train_dapt['Label'].value_counts()
        print(f"      Phân bổ nhãn trong df_train_dapt: \n{label_counts_train_dapt}")
        if any(label_counts_train_dapt < một_ngưỡng_nào_đó): 
            print("      Áp dụng SMOTE cho df_train_dapt...")
            X_train_dapt_features = df_train_dapt.drop(columns=['Label'])
            y_train_dapt_labels = df_train_dapt['Label']
            smote_dapt = SMOTE(random_state=42) # Cấu hình sampling_strategy nếu cần
            X_res_dapt, y_res_dapt = smote_dapt.fit_resample(X_train_dapt_features, y_train_dapt_labels)
            df_train_dapt = pd.DataFrame(X_res_dapt, columns=X_train_dapt_features.columns)
            df_train_dapt['Label'] = y_res_dapt
            print(f"      ✅ SMOTE hoàn tất. df_train_dapt shape mới: {df_train_dapt.shape}")
        else:
            print("      Không cần SMOTE cho df_train_dapt.")

del df_dapt_final, df_dapt_stratify_ready # Giải phóng bộ nhớ
gc.collect()
check_ram()

## 5. Tải Mô hình đã Huấn luyện và Thực hiện Finetuning

Chúng ta sẽ tải mô hình Random Forest và XGBoost đã được huấn luyện trên CSE-CIC-IDS2018.

In [ ]:
print("🔄 Tải các mô hình đã huấn luyện từ CSE-CIC-IDS2018...")
rf_model_pretrained = None
xgb_model_pretrained = None

try:
    RF_MODEL_PATH = os.path.join(PRETRAINED_MODEL_DIR, 'random_forest_model.pkl')
    rf_model_pretrained = joblib.load(RF_MODEL_PATH)
    print(f"✅ Mô hình Random Forest đã tải từ: {RF_MODEL_PATH}")
except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file mô hình Random Forest tại '{RF_MODEL_PATH}'.")
except Exception as e:
    print(f"❌ Lỗi khi tải mô hình Random Forest: {e}")

try:
    # Xác định tên file model XGBoost (gpu_hist hay hist tùy thuộc vào lần chạy trước)
    xgb_model_gpu_path = os.path.join(PRETRAINED_MODEL_DIR, 'xgboost_gpu_hist_model.pkl')
    xgb_model_cpu_path = os.path.join(PRETRAINED_MODEL_DIR, 'xgboost_hist_model.pkl') # Giả sử có thể có file này
    XGB_MODEL_PATH = ''
    if os.path.exists(xgb_model_gpu_path):
        XGB_MODEL_PATH = xgb_model_gpu_path
    elif os.path.exists(xgb_model_cpu_path):
        XGB_MODEL_PATH = xgb_model_cpu_path
    else:
        print(f"❌ Lỗi: Không tìm thấy file mô hình XGBoost (gpu_hist hoặc hist) trong '{PRETRAINED_MODEL_DIR}'.")
    
    if XGB_MODEL_PATH:
        xgb_model_pretrained = joblib.load(XGB_MODEL_PATH)
        print(f"✅ Mô hình XGBoost đã tải từ: {XGB_MODEL_PATH}")
except FileNotFoundError: # Bắt lỗi cụ thể nếu XGB_MODEL_PATH vẫn rỗng
     pass # Đã xử lý ở trên
except Exception as e:
    print(f"❌ Lỗi khi tải mô hình XGBoost: {e}")

# Chuẩn bị dữ liệu DAPT2020 cho huấn luyện (nếu df_train_dapt và df_test_dapt đã được tạo)
if 'df_train_dapt' in locals() and not df_train_dapt.empty:
    X_train_dapt_np = df_train_dapt.drop(columns=['Label']).values
    y_train_dapt_np = df_train_dapt['Label'].values
else:
    print("❌ Dữ liệu huấn luyện DAPT2020 (df_train_dapt) chưa sẵn sàng.")
    # exit()

if 'df_test_dapt' in locals() and not df_test_dapt.empty:
    X_test_dapt_np = df_test_dapt.drop(columns=['Label']).values
    y_test_dapt_np = df_test_dapt['Label'].values
else:
    print("❌ Dữ liệu kiểm thử DAPT2020 (df_test_dapt) chưa sẵn sàng.")
    # exit()

# Lấy class_names từ label_mapping của CSE-CIC-IDS2018 (giả định DAPT2020 đã được map vào các lớp này)
if 'label_mapping_cse' in locals():
    class_names_for_eval = [k for k, v in sorted(label_mapping_cse.items(), key=lambda item: item[1])]
    num_classes_for_eval = len(class_names_for_eval)
else:
    print("❌ Thiếu label_mapping_cse. Không thể xác định tên lớp để đánh giá.")
    # Tạo class_names mặc định nếu cần để tránh lỗi, nhưng kết quả sẽ khó diễn giải
    max_label_val = 0
    if 'y_train_dapt_np' in locals() and len(y_train_dapt_np) > 0:
        max_label_val = max(max_label_val, y_train_dapt_np.max())
    if 'y_test_dapt_np' in locals() and len(y_test_dapt_np) > 0:
        max_label_val = max(max_label_val, y_test_dapt_np.max())
    class_names_for_eval = [f"Class_{i}" for i in range(int(max_label_val) + 1)]
    num_classes_for_eval = len(class_names_for_eval)

clean_ram()

### 5.1 Finetuning/Huấn luyện lại Random Forest trên DAPT2020

Random Forest không dễ để "finetune" bằng cách huấn luyện tiếp. Thay vào đó, chúng ta sẽ huấn luyện một mô hình RF mới trên DAPT2020, nhưng có thể sử dụng lại các siêu tham số tốt từ mô hình trước đó nếu muốn.

In [ ]:
if 'X_train_dapt_np' in locals() and rf_model_pretrained is not None:
    print("\n" + "="*10 + " 🌳 Finetuning/Huấn luyện lại Random Forest trên DAPT2020 " + "="*10)
    
    # Lấy các tham số từ mô hình đã huấn luyện trước (nếu muốn sử dụng lại cấu hình)
    rf_params = rf_model_pretrained.get_params()
    # Có thể bạn muốn giảm n_estimators hoặc max_depth khi finetuning trên tập nhỏ hơn
    # hoặc nếu DAPT2020 rất khác biệt và bạn muốn mô hình học lại nhiều hơn.
    # rf_params['n_estimators'] = 50 # Ví dụ: giảm số cây
    # rf_params['warm_start'] = True # Cho phép thêm cây vào mô hình hiện có - KHÔNG phải lúc nào cũng hiệu quả
                                    # và thường thì huấn luyện lại từ đầu với cấu hình tốt sẽ ổn định hơn.
                                    # Đối với sklearn Random Forest, warm_start chỉ hoạt động khi fit lại trên cùng dữ liệu
                                    # hoặc khi tăng n_estimators. Nó không thực sự "học tiếp" trên dữ liệu mới.
                                    # Do đó, chúng ta sẽ huấn luyện một mô hình mới với các tham số đó.
    
    rf_model_dapt_tuned = RandomForestClassifier(**rf_params) # Sử dụng các tham số đã lấy
    # Hoặc định nghĩa lại hoàn toàn:
    # rf_model_dapt_tuned = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)

    print(f"   - Khởi tạo Random Forest cho DAPT2020 với params: {rf_model_dapt_tuned.get_params().get('n_estimators', 'N/A')} estimators, max_depth {rf_model_dapt_tuned.get_params().get('max_depth', 'N/A')}")

    print("   - Đang huấn luyện Random Forest trên DAPT2020...")
    start_rf_dapt_train_time = time.time()
    try:
        rf_model_dapt_tuned.fit(X_train_dapt_np, y_train_dapt_np)
        rf_dapt_train_time = time.time() - start_rf_dapt_train_time
        print(f"   ✅ Huấn luyện Random Forest trên DAPT2020 hoàn thành trong {rf_dapt_train_time:.2f} giây.")

        # Lưu mô hình đã finetune
        rf_dapt_tuned_model_path = os.path.join(OUTPUT_DIR, "random_forest_model_finetuned_dapt2020.pkl")
        joblib.dump(rf_model_dapt_tuned, rf_dapt_tuned_model_path)
        print(f"   💾 Mô hình Random Forest đã finetune được lưu vào: '{rf_dapt_tuned_model_path}'")

        # Đánh giá
        if 'X_test_dapt_np' in locals():
            evaluate_model_finetuned(rf_model_dapt_tuned, X_test_dapt_np, y_test_dapt_np, class_names_for_eval, "Random Forest (Finetuned DAPT2020)", OUTPUT_DIR)
        else:
            print("   ⚠️ Không có dữ liệu kiểm thử DAPT2020 để đánh giá Random Forest.")

    except Exception as e:
        print(f"   ❌ Lỗi khi huấn luyện/đánh giá Random Forest trên DAPT2020: {e}")
    
    del rf_model_dapt_tuned
    gc.collect()
else:
    print("⚠️ Không thể finetune Random Forest: thiếu dữ liệu huấn luyện DAPT2020 hoặc mô hình Random Forest gốc.")

### 5.2 Finetuning XGBoost trên DAPT2020

XGBoost cho phép huấn luyện tiếp một mô hình đã có bằng cách truyền mô hình đó vào tham số `xgb_model` của hàm `fit`.

In [ ]:
if 'X_train_dapt_np' in locals() and xgb_model_pretrained is not None:
    print("\n" + "="*10 + " 🚀 Finetuning XGBoost trên DAPT2020 " + "="*10)
    
    # Lấy các tham số từ mô hình XGBoost đã huấn luyện trước
    # Một số tham số như n_estimators có thể không cần thiết khi huấn luyện tiếp
    xgb_params_pretrained = xgb_model_pretrained.get_params()
    print(f"   - Tham số của XGBoost gốc: {xgb_params_pretrained.get('n_estimators')}, lr: {xgb_params_pretrained.get('learning_rate')}, max_depth: {xgb_params_pretrained.get('max_depth')}")

    # Thiết lập tree_method (gpu_hist hoặc hist) cho finetuning
    tree_method_finetune = 'hist' # Mặc định CPU
    if torch.cuda.is_available():
        tree_method_finetune = 'gpu_hist'
        print(f"   - Sẽ sử dụng 'tree_method': '{tree_method_finetune}' cho finetuning XGBoost.")
    else:
        print(f"   - GPU không khả dụng, sử dụng 'tree_method': '{tree_method_finetune}' (CPU) cho finetuning XGBoost.")

    # Tạo một đối tượng XGBClassifier mới để finetune. 
    # Quan trọng: num_class phải khớp với số lớp trong y_train_dapt_np sau khi đã ánh xạ!
    # Nếu bạn đã ánh xạ nhãn DAPT2020 vào hệ thống N lớp của CSE, thì num_class giữ nguyên.
    # Nếu bạn tạo thêm lớp mới cho DAPT2020 (khó hơn), num_class phải thay đổi.

    current_num_classes_dapt = len(np.unique(y_train_dapt_np)) if 'y_train_dapt_np' in locals() else num_classes_for_eval

    xgb_model_dapt_finetuned = xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=current_num_classes_dapt, # Số lớp trong dữ liệu DAPT2020 đã map
        # Giữ lại các tham số cấu trúc cây quan trọng từ mô hình gốc, hoặc điều chỉnh:
        learning_rate=xgb_params_pretrained.get('learning_rate', 0.1) * 0.5, # Có thể giảm learning rate khi finetuning
        max_depth=xgb_params_pretrained.get('max_depth', 6),
        subsample=xgb_params_pretrained.get('subsample', 0.8),
        colsample_bytree=xgb_params_pretrained.get('colsample_bytree', 0.8),
        gamma=xgb_params_pretrained.get('gamma', 0),
        n_estimators=100, # Số lượng cây mới để thêm vào khi finetuning (điều chỉnh)
        random_state=42,
        use_label_encoder=False, # Đã bỏ từ XGBoost >1.3
        eval_metric='mlogloss',
        n_jobs=-1,
        tree_method=tree_method_finetune
    )

    print(f"   - Đang finetuning XGBoost trên DAPT2020 (bắt đầu từ mô hình đã huấn luyện trên CSE-CIC-IDS2018)...")
    print(f"     Tham số finetuning: lr={xgb_model_dapt_finetuned.learning_rate}, n_est={xgb_model_dapt_finetuned.n_estimators}")
    start_xgb_dapt_train_time = time.time()
    try:
        # Huấn luyện tiếp (finetune)
        xgb_model_dapt_finetuned.fit(
            X_train_dapt_np, 
            y_train_dapt_np, 
            xgb_model=xgb_model_pretrained, # Đây là điểm mấu chốt để finetune
            verbose=False
        )
        xgb_dapt_train_time = time.time() - start_xgb_dapt_train_time
        print(f"   ✅ Finetuning XGBoost trên DAPT2020 hoàn thành trong {xgb_dapt_train_time:.2f} giây.")

        # Lưu mô hình đã finetune
        xgb_dapt_finetuned_model_path = os.path.join(OUTPUT_DIR, f"xgboost_{tree_method_finetune}_model_finetuned_dapt2020.pkl")
        joblib.dump(xgb_model_dapt_finetuned, xgb_dapt_finetuned_model_path)
        print(f"   💾 Mô hình XGBoost đã finetune được lưu vào: '{xgb_dapt_finetuned_model_path}'")

        # Đánh giá
        if 'X_test_dapt_np' in locals():
            evaluate_model_finetuned(xgb_model_dapt_finetuned, X_test_dapt_np, y_test_dapt_np, class_names_for_eval, f"XGBoost ({tree_method_finetune}, Finetuned DAPT2020)", OUTPUT_DIR)
        else:
            print("   ⚠️ Không có dữ liệu kiểm thử DAPT2020 để đánh giá XGBoost.")

    except xgb.core.XGBoostError as e_xgb:
        print(f"   ❌ Lỗi XGBoost khi finetuning trên DAPT2020: {e_xgb}")
        if "CUDA" in str(e_xgb) or "GPU" in str(e_xgb):
            print("      Lỗi có thể liên quan đến cài đặt CUDA/GPU. Thử lại với tree_method='hist' (CPU).")
    except Exception as e:
        print(f"   ❌ Lỗi khi finetuning/đánh giá XGBoost trên DAPT2020: {e}")

    del xgb_model_dapt_finetuned
    gc.collect()
else:
    print("⚠️ Không thể finetune XGBoost: thiếu dữ liệu huấn luyện DAPT2020 hoặc mô hình XGBoost gốc.")

clean_ram()
print("\n🏁🏁🏁 Quá trình finetuning hoàn tất! 🏁🏁🏁")